In [1]:
import tensorflow as tf

### Tensorflow 从文件读取数据
+ 把样本数据写入TFRecords二进制文件；
+ 再从队列中读取

__TFRecords__ 是一种二进制文件，能更好地利用内存，更方便地复制和移动，并且不需要单独的 标记文件


#### 1.生成TFRecords文件

In [4]:
def main(unused_argv):
    #获取数据
    data_sets = mnist.read_data_sets(FLAGS.dictory,
                                     dtype = tf.uint8,
                                     reshape = False,
                                     validation_size = FLAGS.validation_size)
    #将数据转换为tf.train.Example类型并写入TFRecords文件
    convert_to(data_sets.train,'train')
    convert_to(data_sets.validation,'validation')
    convert_to(data_sets.test,'test')
    
def convert_to(data_set,name):
    images = data_set.images
    labels = data_set.labels
    num_examples = data_set.num_examples #55000个训练数据，5000个验证数据，10000个测试数据
    
    if images.shape[0] != num_examples:
        raise ValueError('Images size %d does not match label size %d'%(images.shape[0],num_examples))
    rows = images.shape[1] #28
    cols = images.shape[2] #28
    depth = images.shape[3]#1单通道
    
    filename = os.path.join(FLAGS.directory,name+'.tfrecords')#整合目录创建新目录
    print('Writing',filename)
    writer = tf.python_io.TFRecordWriter(filename)
    
    for index in range(num_examples):
        image_raw = images[index].tostring()
        #写入协议缓冲区中，height、width、depth、label编码成int64类型image_raw编码成二进制类型
        example = tf.train.Example(feature=tf.train.Features(feature={
                                                        'height':_int64_feature(rows),
                                                        'width':_int64_feature(cols),
                                                        'depth':_int64_feature(depth),
                                                        'label':_int64_feature(int(labels[index])),
                                                        'image_raw':_bytes_feature(image_raw)}))
        writer.write(example.SerializeToString())#序列化字符串
    writer.close()

def _int_64_feature(value):#编码函数
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):#编码函数
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

#### 2.从队列中读取

一旦生成TFRecords文件，接下来就可以使用队列读取数据了（主要分三步）：
+ 创建张量，从二进制文件读取一个样本
+ 创建张量，从二进制文件随机读取一个mini-batch
+ 把每一批张量传入网络作为输入节点

In [ ]:
def read_and_decode(filename_queue):
    reader = tf.TFRecordReader()
    _,serialized_example = reader.read(filename_queue)
    #解析example
    features = tf.parse_single_example(serialized_example,
                                       #必须写明features里面的key名称
                                       feature={
                                           'image_raw':tf.FixedLenFeature([],tf.string),#图片是string类型
                                           'label':tf.FixedLenFeature([],tf.int64)
                                       })
    # 对于BytesList，要重新进行解码，把 string 类型的 0 维 Tensor 变成 uint8 类型的一维 Tensor
    image = tf.decode_raw(features['image_raw'],tf.uint8)
    image.set_shape([mnist.Image_PIXELS])
    # 
    
    